In [ ]:
import numpy as np
from keras.models import load_model
from keras import backend as K
from keras.models import Model
import matplotlib.pyplot as plt
import cv2
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
import os
import yaml
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from utils import *

In [ ]:
def Visual_feature(fea, num=4):
    #input:(512,512,64)
    dimension = fea.shape[-1]
    if dimension>=16:
        num=4
    elif dimension<16 and dimension>=9:
        num=3
    elif dimension<9:
        num=2
    plt.figure(figsize=(32,32))
    index = 1
    for i in range(1,num+1):
        for j in range(1,num+1):
            plt.subplot(num,num,index)
            index+=1
            w = fea.shape[-2]
            img = fea[:,:,index].reshape([w,w,1])
            img = np.concatenate((img,img,img),axis=-1)
            plt.imshow(img)
def Visual_image(img,isgray=True):
    plt.figure(figsize=(8, 8))
    if isgray:
        img = np.concatenate((img,img,img),axis=-1)
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)

In [ ]:
with open("config.yaml") as pfile:
    cfgs = yaml.load(pfile)
dl = DataLoader(cfgs)
# 'stage4_unit1_relu1', 'stage3_unit1_relu1', 'stage2_unit1_relu1', 'relu0'
model = Unet(backbone_name='resnet50', encoder_weights='imagenet')
model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])
model.load_weights(os.path.join(cfgs["SAVE_DIR"],"weights","epoch295.h5"))
print("RETORE SUCCESSFULLY!")
test_images, test_ulabels,_,_,_= dl.get_test_data()
#results = model.predict(test_images, batch_size=10, verbose=1)

In [ ]:
layer_model = Model(inputs=model.input,
                         outputs=model.get_layer('fuse_dir').output)
layer_result = layer_model.predict(test_images, batch_size=10, verbose=1)
print(layer_result.shape)

In [ ]:

Visual_image(layer_result[5])
# Visual_feature(layer_result[0])